In [1]:
output_names = ['Predictions/Softmax']
input_names = ['input_4']

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions

/home/cwc-work/python-envs/env3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cwc-work/python-envs/env3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cwc-work/python-envs/env3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cwc-work/python-envs/env3.6/lib/python3.6/s

In [2]:
classification_graph = tf.Graph()
def get_frozen_graph(graph_file):
    """Read Frozen Graph file from disk."""
    with classification_graph.as_default():
        graph_def = tf.GraphDef()
        with tf.gfile.FastGFile(graph_file, "rb") as f:
            graph_def.ParseFromString(f.read())
    return graph_def

In [3]:
tff_graph = get_frozen_graph('./Saved-Model/Inception-v1-tf-graph.pb')

# Create session and load graph
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_sess = tf.Session(config=tf_config)
tf.import_graph_def(tff_graph, name='')

Instructions for updating:
Use tf.gfile.GFile.


In [4]:
# Get graph input size
for node in tff_graph.node:
    if 'input_' in node.name:
        size = node.attr['shape'].shape
        image_size = [size.dim[i].size for i in range(1, 4)]
        break
print("image_size: {}".format(image_size))

# input and output tensor names.
input_tensor_name = input_names[0] + ":0"
output_tensor_name = output_names[0] + ":0"

print("input_tensor_name: {}\noutput_tensor_name: {}".format(input_tensor_name, output_tensor_name))

output_tensor = tf_sess.graph.get_tensor_by_name(output_tensor_name)

image_size: [224, 224, 3]
input_tensor_name: input_4:0
output_tensor_name: Predictions/Softmax:0


In [6]:
# Optional image to test model prediction.
img_path = './elephant.jpg'

img = image.load_img(img_path, target_size=(100,100))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

feed_dict = {
    input_tensor_name: x
}

preds = tf_sess.run(output_tensor, feed_dict)

# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds[:, 1:], top=3)[0])

ValueError: Cannot feed value of shape (1, 100, 100, 3) for Tensor 'input_4:0', which has shape '(?, 224, 224, 3)'